<img src="https://easy.gems.dkrz.de/_images/landing-page-banner.png"
     width="30%"
     alt="HEALPix logo"
     align="right"
/>

# easy.gems for HEALPix Analysis & Visualization

### In this section, you'll learn:

* Utilizing intake to open a HEALPix data catalog
* Using the `healpix` package to perform HEALPix operations to look at basic statistics
* Plotting HEALPix data via easy.gems functionality

### Related Documentation

* [Getting started with hierarchical HEALPix data - easy.gems Documentation](https://easy.gems.dkrz.de/Processing/healpix/healpix_starter.html#)
* [Plotting with cartopy - easy.gems Documentation](https://easy.gems.dkrz.de/Processing/healpix/healpix_cartopy.html)
* [Intake Cookbook](https://projectpythia.org/intake-cookbook/README.html)

### Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [HEALPix overview](00-healpix) | Necessary  | |

**Time to learn**: 30 minutes

-----

## Open data catalog

:::{tip} If you think you first need to learn about Intake, Pythia's [Intake Cookbook](https://projectpythia.org/intake-cookbook/README.html) is a great resource to do so.:::

Let us open a HEALPix data catalog using `intake` and read in a `Xarray.Dataset` (i.e. output of the nextGEMS Cycle 3 `ICON` run `ngc3028`):

In [ ]:
import intake

cat = intake.open_catalog("https://data.nextgems-h2020.eu/online.yaml")
model_run = cat.ICON.ngc3028

:::{tip} We highly recommend checking out the [easy.gems documentation on HEALPix data catalogs](https://easy.gems.dkrz.de/Processing/healpix/healpix_starter.html#the-catalog) to understand the `zoom` and `time` parametrization of the model runs in the catalogs. In short, each model run has an independent `Dataset` for each combination of the `zoom` and `time` parameters as depicted below (image credit [easy.gems documentation on HEALPix data catalogs](https://easy.gems.dkrz.de/Processing/healpix/healpix_starterhtml#the-catalog)):::

<img src="https://easy.gems.dkrz.de/_images/ds_hierarchy.png"
     width=600
     alt="Matplotlib Logo"
/>

## Explore datasets

So, the coarsest dataset in this model run would be as follows (Even if we called it without specifying any parameters, i.e. `model_run.to_dask()`, the result would be same as the `ds_coarsest` below since the model run defaults to the coarsest settings.):

In [ ]:
ds_coarsest = model_run(zoom=0, time="P1D").to_dask()
ds_coarsest

Now, let us look at a dataset with finer zoom level still with the coarsest time and another dataset with a finer zoom level with the finest time (which may be useful for daily analyses) dataset:

In [ ]:
ds_fine = model_run(zoom=7).to_dask()
ds_fine

In [ ]:
ds_finesttime = model_run(zoom=6, time="PT30M").to_dask()
ds_finesttime

## HEALPix basic stats with the`healpix` package

Let us look at the global and Boulder, CO, USA temperature averages for the temporal span of the dataset. 

For this, we will first need to define a few HEALPix helper functions to get the `nest` and `nside` values from the dataset, then find the HEALPix pixel that Boulder coords fall in, and finally plot those temporal averages using `matplotlib`.

In [ ]:
import healpix as hp
import matplotlib.pylab as plt

### HEALPix helper functions

In [ ]:
def get_nest(ds):
    return ds.crs.healpix_order == "nest"
    
def get_nside(ds):
    return ds.crs.healpix_nside

### HEALPix pixel containing Boulder's coords

In [ ]:
%%time
boulder_lon = -105.2747
boulder_lat = 40.0190

boulder_pixel = hp.ang2pix(
    get_nside(ds_fine), boulder_lon, boulder_lat, lonlat=True, nest=get_nest(ds_fine)
)

### Global and Boulder's temperature averages

In [ ]:
%%time
ds_fine.tas.isel(cell=boulder_pixel).plot(label="Boulder")

ds_coarsest.tas.mean("cell").plot(label="Global mean")

plt.legend()

## Plotting with easy.gems and cartopy

 In this part, we will look at the ` healpix_show` function that is provided by `easy.gems` for convenient HEALPix plotting.

In [ ]:
import easygems.healpix as eghp

import cartopy.crs as ccrs
import cartopy.feature as cf
import cmocean

### Global plots

Most of this is `matplotlib` and `cartopy` code, but have a look at how `eghp.healpix_show()` is called. The following code will plot global temperature (at the first timestep for simplicity)

In [ ]:
projection = ccrs.Robinson(central_longitude=-135.5808361)

fig, ax = plt.subplots(
    figsize=(8, 4), subplot_kw={"projection": projection}, constrained_layout=True
)

ax.set_global()

eghp.healpix_show(ds_fine.tas.isel(time=0), ax=ax, cmap=cmocean.cm.thermal)

ax.add_feature(cf.COASTLINE, linewidth=0.8)
ax.add_feature(cf.BORDERS, linewidth=0.4)

### Regional plots

If plotting a region of interest is desired, it is also possible through setting extents of the `matplotlib` plot. 

Let us look into USA map using the Boulder, CO, USA coords we had used before for simplicity:

In [ ]:
projection = ccrs.Robinson(central_longitude=boulder_lon)

fig, ax = plt.subplots(
    figsize=(8, 4), subplot_kw={"projection": projection}, constrained_layout=True
)
ax.set_extent([boulder_lon-20, boulder_lon+40, boulder_lat-20, boulder_lat+10], crs=ccrs.PlateCarree())

eghp.healpix_show(ds_fine.tas.isel(time=0), ax=ax, cmap=cmocean.cm.thermal)

ax.add_feature(cf.COASTLINE, linewidth=0.8)
ax.add_feature(cf.BORDERS, linewidth=0.4)

## Further `easy.gems` and `healpix`

These are only a sampling of HEALPix and easy.gems capabilities; if you are interested in learning more, be sure to check out the usage examples at the [easy.gems HEALPix Documentation](https://easy.gems.dkrz.de/Processing/healpix/index.html#).

## What is next?

The [next section](02-uxarray) will provide an UXarray workflow that loads in and analyzes & visualizes HEALPix data.